### 其实所谓的prompt(提示词)就是你的输入
```bash
"你的问题"
```

### template(模板)也就是一个提示，给予模型某些定义，可以这么理解"


```bash
"you are a chat bot,was named phb, answer my questions :{question}".format(question="你的问题") #template
```

### 而parser(解析器)也是一种提示，提示模型怎么回答，可以这么理解


```bash
"you are a chat bot,was named phb, answer my questions :{question}".format(question="你的问题") #template

请使用一下格式回答问题 #parser
{
    question:
    answer:
}
```
### 当然，作为parser，后续对回答也有相应的解析

## 不懂看代码


### 最简prompt_template

In [12]:
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
llm=OpenAI(model="gpt-3.5-turbo-instruct",temperature=1)

template=PromptTemplate.from_template("you are a chat bot,was named phb, answer my questions :{question}")
#template=PromptTemplate(template="you are a chat bot,named phb,answer my questions :{question}",input_variables=["qustion"])

In [13]:
template
prompt=template.format(question="who are you?")
print(template)
print(prompt)

input_variables=['question'] template='you are a chat bot,was named phb, answer my questions :{question}'
you are a chat bot,was named phb, answer my questions :who are you?


In [14]:
llm.invoke(prompt)

'\n\nI am a pre-programmed artificial intelligence designed to communicate and assist users with various tasks and information. My name is phb. How can I assist you?'

### 最简ChatPromptTemplate

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=1)

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is PHB."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

# ChatPromptTemplate(
#     input_variables=['user_input'], 
#     messages=[
#         SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful AI bot. Your name is PHB.')),
#         HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Hello, how are you doing?')), 
#         AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="I'm doing well, thanks!")), 
#         HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))
#     ]
# )



messages = chat_template.format_messages(user_input="What is your name?")

In [28]:
chat_template

ChatPromptTemplate(input_variables=['user_input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful AI bot. Your name is PHB.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Hello, how are you doing?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="I'm doing well, thanks!")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))])

In [29]:
messages

[SystemMessage(content='You are a helpful AI bot. Your name is PHB.'),
 HumanMessage(content='Hello, how are you doing?'),
 AIMessage(content="I'm doing well, thanks!"),
 HumanMessage(content='What is your name?')]

In [33]:
llm.invoke(messages)


AIMessage(content='My name is PHB. How can I assist you today?')

In [34]:
print(llm.invoke(messages).content)

My name is PHB. How can I assist you today?


### Few-shot prompt templates


In [22]:
from langchain.prompts import PromptTemplate,FewShotPromptTemplate


In [24]:
examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

In [28]:
example_template =PromptTemplate(template="question: {question}\nanswer: {answer}",
                                 input_variables=["question","answer"])
print(example_template.format(**examples[0]))


question: Who lived longer, Muhammad Ali or Alan Turing?
answer: 
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



In [29]:
prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_template,
    suffix="Question: {input}",
    input_variables=["input"],
)

In [32]:
prompt=prompt_template.format(input="Who was the maternal grandfather of George Washington?")

In [33]:
prompt

'question: Who lived longer, Muhammad Ali or Alan Turing?\nanswer: \nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n\n\nquestion: When was the founder of craigslist born?\nanswer: \nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n\n\nquestion: Who was the maternal grandfather of George Washington?\nanswer: \nAre follow up questions needed here: Yes.\nFollow up: Who was the mother of George Washington?\nIntermediate answer: The mother of George Washington was Mary Ball Washing

In [34]:
llm.invoke(prompt)

'\nAre follow up questions needed here: Yes.\nFollow up: Who was the mother of George Washington?\nIntermediate answer: The mother of George Washington was Mary Ball Washington.\nFollow up: Who was the father of Mary Ball Washington?\nIntermediate answer: The father of Mary Ball Washington was Joseph Ball.\nFollow up: Who was the father of Joseph Ball?\nIntermediate answer: The father of Joseph Ball was William Ball.\nSo the final answer is: William Ball'

### Output Parsers

In [1]:
from langchain.output_parsers import PydanticOutputParser,CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

llm=OpenAI(model="gpt-3.5-turbo-instruct",temperature=1)



In [2]:
output_parser=CommaSeparatedListOutputParser()
format_instructions=output_parser.get_format_instructions()

In [3]:
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [4]:
template=PromptTemplate(template="you are a chat bot,was named phb, answer my questions :{question}/n{format_instructions}",
                        input_variables=["question"],
                        partial_variables={"format_instructions":format_instructions})

In [5]:

#为什么会报错
prompt=template.format(question="为我的花店取十个中文名字")
print(prompt)

you are a chat bot,was named phb, answer my questions :为我的花店取十个中文名字/nYour response should be a list of comma separated values, eg: `foo, bar, baz`


In [6]:
result=llm.invoke(prompt)
print(result)


1. 花语缤纷 
2. 花繁落尽 
3. 花好月圆 
4. 花影流香 
5. 花香四溢 
6. 花开花落 
7. 花间一梦 
8. 花海如画 
9. 花前月下 
10. 花开富贵


In [8]:
output_parser.parse(result)

['1. 花语缤纷 \n2. 花繁落尽 \n3. 花好月圆 \n4. 花影流香 \n5. 花香四溢 \n6. 花开花落 \n7. 花间一梦 \n8. 花海如画 \n9. 花前月下 \n10. 花开富贵']